In [1]:
from utils import *

In [2]:
from azureml.core import Workspace

# Configure experiment
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
# Create or get training cluster
aml_cluster = get_aml_cluster(ws, cluster_name="cpu-cluster")
aml_cluster.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# Create a run configuration
run_conf = get_run_config(['numpy', 'pandas', 'scikit-learn', 'tensorflow'])

In [5]:
from azureml.core.dataset import Dataset

dataset = Dataset.get_by_name(ws, name='titanic')
data_in = dataset.as_named_input('titanic')

In [6]:
from azureml.pipeline.steps import PythonScriptStep

step = PythonScriptStep(name='Preprocessing',
                        script_name="preprocess_input.py",
                        source_directory="code",
                        arguments=["--input", data_in],
                        inputs=[data_in],
                        runconfig=run_conf,
                        compute_target=aml_cluster)

In [7]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(ws, steps=[step])

In [8]:
pipeline.validate()

Step Preprocessing is ready to be created [e2e63986]


[]

In [9]:
from azureml.core import Experiment

exp = Experiment(ws, "azureml-pipeline")
run = exp.submit(pipeline)

Created step Preprocessing [e2e63986][e24fc276-1bbb-4b32-b0d4-9e3f07074189], (This step will run and generate new outputs)
Submitted PipelineRun 64a99900-b260-44c7-85a3-64c7fe467d01
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/64a99900-b260-44c7-85a3-64c7fe467d01?wsid=/subscriptions/21dc412b-d9eb-42e7-8317-55bc8eb10cf5/resourcegroups/packt/workspaces/mldemows&tid=e702ab66-1ac1-42ba-b41b-8195d943a1e0


In [10]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …